# DIMS.FARM REST API example

In [1]:
import os
import json

import pandas as pd
from requests import HTTPError

from watobs.datafarm import DatafarmRepository

api_key = os.getenv("DATAFARM_API_KEY")
assert api_key is not None


### Connect to DIMS.FARM REST API

In [2]:
dfr = DatafarmRepository(api_key)
dfr.connect()

### List available time series

In [3]:
time_series_list = dfr.list_time_series()
time_series_list.head()

,ID,EntityID,Touched,IDName,IDDescription,LocationID,TimeSeriesParameterID,TimeSeriesMediaID,TimeSeriesUnitID,TimeSeriesDataTypeID,TimeSeriesDatasourceID,TimeSeriesTypeID,TimeSeriesStatusID,TimeSeriesDataProviderID,TimeSeriesDataProviderArgument,DataExpectedCount,DataArchiveAfter,DataDeleteAfter
GUID,,,,,,,,,,,,,,,,,,
{62F60AF2-C34A-11ED-B2F7-1831BF2DC749},2,AKZ_waves_CMEMS_unfiltered_Hm0,2023-03-20 17:18:42,AKZ_waves_CMEMS_unfiltered_Hm0,Spectral significant wave height,AKZ,Hm0,waves,m,Provider,,CMEMS,unfiltered,,,NaN,NaN,NaN
{7394526A-C34A-11ED-B2F7-1831BF2DC749},3,AKZ_waves_CMEMS_unfiltered_Tz,2023-03-20 17:18:43,AKZ_waves_CMEMS_unfiltered_Tz,Average zero crossing wave period (Tz),AKZ,Tz,waves,s,Provider,,CMEMS,unfiltered,,,NaN,NaN,NaN
{7394526B-C34A-11ED-B2F7-1831BF2DC749},4,Bor1_currents_RVO-FUGRO_derived_CD,2023-03-20 17:18:43,Bor1_currents_RVO-FUGRO_derived_CD,Current direction,Bor1,CD,currents,degree-N-to,Provider,,RVO-FUGRO,derived,,,NaN,NaN,NaN
{7394526C-C34A-11ED-B2F7-1831BF2DC749},5,Bor1_currents_RVO-FUGRO_derived_CS,2023-03-20 17:18:44,Bor1_currents_RVO-FUGRO_derived_CS,Current speed,Bor1,CS,currents,m/s,Provider,,RVO-FUGRO,derived,,,NaN,NaN,NaN
{7394526D-C34A-11ED-B2F7-1831BF2DC749},6,Bor1_currents_RVO-FUGRO_missing_CD-4,2023-03-20 17:18:45,Bor1_currents_RVO-FUGRO_missing_CD-4,Current direction,Bor1,CD,currents,degree-N-to,Provider,,RVO-FUGRO,missing,,,NaN,NaN,NaN


### Get time series data

In [4]:
data = dfr.get_data(
    time_series_id="testapi.insert",
    limit=5,
    ascending=True
)
data

,Data,QualityTxt
RefDateTimeRef,,
2013-05-22 10:37:41,0.105753,ok
2014-10-22 10:37:41,0.404689,ok
2022-09-28 20:32:31,637.000000,ok
2022-09-28 20:32:31,637.000000,ok
2022-09-28 20:32:36,637.000000,ok


### Delete data in a given range:

In [5]:
start, end = data.index[0], data.index[-1]
print(f"Deleting data in range [{start}, {end})")
dfr.delete_data("testapi.insert", start=start, end=end)

Deleting data in range [2013-05-22 10:37:41, 2022-09-28 20:32:36)


<Response [200]>

Check that the data has been deleted:

In [6]:
data = dfr.get_data(
    time_series_id="testapi.insert",
    limit=10,
    ascending=True
)
data

,Data,QualityTxt
RefDateTimeRef,,
2022-09-28 20:32:36,637.0,ok
2022-09-28 21:02:29,637.0,ok
2022-09-28 21:02:29,637.0,ok
2022-09-28 21:03:22,637.0,ok
2022-09-28 21:04:27,317.0,ok
2022-09-28 21:15:23,637.0,ok
2022-09-28 21:56:41,637.0,ok
2022-09-28 21:56:51,637.0,ok
2022-09-28 21:56:56,637.0,ok


### Insert time series data

Here we prepare some data to insert in form of a dataframe.

In [9]:
import random

random_date = lambda: pd.Timestamp.now() - pd.Timedelta(days=3000 + random.randint(1, 1000))
rows = 2
new_data = pd.DataFrame({
    "TimeStamp": [random_date() for _ in range(rows)],
    "Data": [random.random() for _ in range(rows)],
    "Quality": ["ok"] * rows,
})
new_data


,TimeStamp,Data,Quality
0,2014-09-08 10:39:24.229608,0.446695,ok
1,2014-05-01 10:39:24.229734,0.617950,ok


We can peek at the json body for the call to the API. The NaN value is represented as {"N": 1, "V": 0}.

In [10]:
body = dfr._get_insert_data_body("testapi.insert", new_data, bulk_insert=True)
body_json = json.dumps(body, indent=2)
print(body_json)

{
  "BulkInsert": true,
  "TimeSeriesName": "testapi.insert",
  "TimeStamp": [
    "2014-09-08T10:39:24.229",
    "2014-05-01T10:39:24.229"
  ],
  "Data": [
    {
      "N": 0,
      "V": 0.4466948011874553
    },
    {
      "N": 0,
      "V": 0.6179502362631356
    }
  ],
  "QualityLevel": [
    0,
    0
  ]
}


To insert the data we use the `insert_data` method:

In [11]:
try:
    res = dfr.insert_data("testapi.insert", new_data, bulk_insert=True)  # TODO: Error 500 ??
except HTTPError as e:
    print(e)

In [13]:
res.json()

{'ItemsWritten': 2}

### Close connection

In [11]:
dfr.close()

# Now this will fail:
try:
    dfr.list_time_series()
except HTTPError as e:
    print(e)

401 Client Error: Unauthorized for url: https://apidevtest.datafarm.work/api/List/TimeSeries/


In [12]:
# Reconnect and try again
dfr.connect()
dfr.list_time_series().head()
dfr.close()

### Using the context manager

In [13]:

with DatafarmRepository(api_key) as dfr:
    data = dfr.get_data(
        time_series_id="testapi.insert",
        limit=2
    )
data

,Data,QualityTxt
RefDateTimeRef,,
2012-08-14 09:58:24,0.727511,ok
2014-12-15 09:58:24,0.024472,ok
